In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [3]:
df = pd.read_csv("Churn_Modelling.csv")

In [4]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
# preprocessing the dataset
df = df.drop(["RowNumber", "CustomerId", "Surname"], axis=1)


In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
# encode categorical variables
label_encoder_gender = LabelEncoder()
df["Gender"] = label_encoder_gender.fit_transform(df["Gender"])


In [8]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
# one-hot encode the Geography column
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder_geo = OneHotEncoder(sparse_output=False)
geo_encoded = one_hot_encoder_geo.fit_transform(df[["Geography"]])

In [ ]:
geo_encoded_df = pd.DataFrame(
    geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(["Geography"])
)

In [ ]:
# combine the encoded Geography columns with the original dataframe
df = pd.concat([df.drop("Geography", axis=1), geo_encoded_df], axis=1)

In [12]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [ ]:
# divide the dataset into features and target variable
X = df.drop("EstimatedSalary", axis=1)
y = df["EstimatedSalary"]

In [ ]:
# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
# save the encoders to a pickle file
with open("label_encoder_gender_regression.pkl", "wb") as f:
    pickle.dump(label_encoder_gender, f)

with open("one_hot_encoder_geo_regression.pkl", "wb") as f:
    pickle.dump(one_hot_encoder_geo, f)
    
# save the scaler to a pickle file
with open("scaler_regression.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [17]:
# ANN for regression problem 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 12), (2000, 12), (8000,), (2000,))

In [20]:
# build the ANN model
model = Sequential(
    [
        Dense(
            64, activation="relu", input_shape=(12,)
        ),  # Hidden layer 1 with 64 neurons
        Dense(32, activation="relu"),  # Hidden layer 2 with 32 neurons
        Dense(1),  # Output layer for regression
    ]
)


In [ ]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [27]:
# compile the model
model.compile(
    optimizer="adam", loss="mean_absolute_error", metrics=["mae"]
)

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
import datetime
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping

# setup the tensorboard
log_dir = "regression/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [33]:
# setup early stopping
early_stopping_callback = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

In [34]:
# train the model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[tensorboard_callback, early_stopping_callback],
)


Epoch 1/100
250/250 [==============================] - 2s 7ms/step - loss: 49665.2812 - mae: 49665.2812 - val_loss: 50354.3555 - val_mae: 50354.3555
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 49654.0273 - mae: 49654.0273 - val_loss: 50350.3906 - val_mae: 50350.3906
Epoch 3/100
250/250 [==============================] - 2s 6ms/step - loss: 49642.9102 - mae: 49642.9102 - val_loss: 50351.4258 - val_mae: 50351.4258
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 49629.4531 - mae: 49629.4531 - val_loss: 50344.3164 - val_mae: 50344.3164
Epoch 5/100
250/250 [==============================] - 2s 8ms/step - loss: 49617.6719 - mae: 49617.6719 - val_loss: 50340.6836 - val_mae: 50340.6836
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 49608.8633 - mae: 49608.8633 - val_loss: 50345.3203 - val_mae: 50345.3203
Epoch 7/100
250/250 [==============================] - 3s 11ms/step - loss: 49598.6289 - mae: 49598.6289 -

In [ ]:
# load tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir regression/logs/fit
